In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AmazonApparelAnalysis") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

print("SparkSession initialized successfully!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/02 14:50:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession initialized successfully!


In [2]:
data = [("Alice", 28), ("Bob", 33), ("Cathy", 25)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)
df.show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 28|
|  Bob| 33|
|Cathy| 25|
+-----+---+



In [18]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("DatasetSummary").getOrCreate()

# Load the dataset (replace with your file path)
file_path = "final_combined_dataset.csv"  # Update with the correct file path
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Display the schema
print("Schema of the Dataset:")
data.printSchema()

# Display the number of rows
print(f"Number of Rows in the Dataset: {data.count()}")

# Display column statistics (numerical only)
print("Summary Statistics:")
data.describe().show()

# Display a few sample rows
print("Sample Rows:")
data.show(5, truncate=False)


24/12/02 14:58:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Schema of the Dataset:
root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- demographic_group: string (nullable = true)
 |-- apparel_section: string (nullable = true)
 |-- category: string (nullable = true)



Number of Rows in the Dataset: 5906333
Summary Statistics:


+-------+-----------+--------------------+--------------+-------------------+--------------------+--------------------+----------------+------------------+------------------+------------------+-------+-----------------+--------------------+--------------------+-----------------+----------------+--------+
|summary|marketplace|         customer_id|     review_id|         product_id|      product_parent|       product_title|product_category|       star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|     review_headline|         review_body|demographic_group| apparel_section|category|
+-------+-----------+--------------------+--------------+-------------------+--------------------+--------------------+----------------+------------------+------------------+------------------+-------+-----------------+--------------------+--------------------+-----------------+----------------+--------+
|  count|    5906333|             5906333|       5906333|            5906333|     

In [4]:
from pyspark.sql.functions import col

# Drop rows with missing critical values
cleaned_data = data.dropna(subset=["customer_id", "product_id", "review_body", "star_rating"])

# Show summary of cleaned data
print(f"Total rows after cleaning: {cleaned_data.count()}")
cleaned_data.show(5, truncate=False)

Total rows after cleaning: 5905272
+-----------+-----------+--------------+----------+--------------+------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
#Text preprocessed code
from pyspark.sql.functions import lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover

# Clean review text: lowercase and remove punctuation
processed_data = cleaned_data.withColumn(
    "cleaned_review",
    lower(regexp_replace("review_body", "[^a-zA-Z\\s]", ""))
)

# Tokenize the text
tokenizer = Tokenizer(inputCol="cleaned_review", outputCol="tokens")
tokenized_data = tokenizer.transform(processed_data)

# Remove stopwords
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
preprocessed_data = remover.transform(tokenized_data)

# Show cleaned and tokenized reviews
preprocessed_data.select("review_body", "cleaned_review", "filtered_tokens").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
from pyspark.sql.functions import udf, col  # Import necessary functions
from pyspark.sql.types import StringType
from pyspark.ml.feature import CountVectorizer, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Step 1: Create Sentiment Labels
def assign_sentiment(rating):
    if rating >= 4:
        return "Positive"
    elif rating == 3:
        return "Neutral"
    else:
        return "Negative"

# Register the UDF
sentiment_udf = udf(assign_sentiment, StringType())
sentiment_data = preprocessed_data.withColumn("sentiment", sentiment_udf(col("star_rating")))

# Step 2: Vectorize Tokenized Text
vectorizer = CountVectorizer(inputCol="filtered_tokens", outputCol="features")

# Step 3: Index Sentiment Labels
label_indexer = StringIndexer(inputCol="sentiment", outputCol="label")

# Step 4: Logistic Regression Classifier
classifier = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# Step 5: Pipeline for Processing
pipeline = Pipeline(stages=[vectorizer, label_indexer, classifier])

# Step 6: Split Data into Training and Testing Sets
train_data, test_data = sentiment_data.randomSplit([0.8, 0.2], seed=42)

# Step 7: Train the Model
model = pipeline.fit(train_data)

# Step 8: Evaluate the Model
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

24/12/02 14:52:00 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:26 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:27 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/02 14:52:27 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:56 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:56 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:57 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:57 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:57 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:58 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:58 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
24/12/02 14:52:59 WARN DAGSchedul

Accuracy: 0.8329615758338705


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lit, when
from pyspark.sql.types import StringType
from pyspark.ml.feature import CountVectorizer, StringIndexer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("AmazonApparelAnalysis") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print("SparkSession initialized successfully!")

# Step 2: Load and Preprocess Data
data_path = "final_combined_dataset.csv"  # Replace with your dataset path
raw_data = spark.read.csv(data_path, header=True, inferSchema=True)

# Handle null values in 'review_body' by replacing them with an empty string
cleaned_data = raw_data.withColumn(
    "review_body",
    when(col("review_body").isNull(), lit("")).otherwise(col("review_body"))
)

# Handle null values in 'star_rating' if needed
cleaned_data = cleaned_data.withColumn(
    "star_rating",
    when(col("star_rating").isNull(), lit(0)).otherwise(col("star_rating"))
)

# Step 3: Define UDF for Sentiment Assignment
def assign_sentiment(rating):
    if rating >= 4:
        return "Positive"
    elif rating == 3:
        return "Neutral"
    else:
        return "Negative"

sentiment_udf = udf(assign_sentiment, StringType())

# Add the sentiment column
sentiment_data = cleaned_data.withColumn("sentiment", sentiment_udf(col("star_rating")))

# Step 4: Tokenize Text Data
tokenizer = Tokenizer(inputCol="review_body", outputCol="filtered_tokens")

# Step 5: Vectorize Text Data
vectorizer = CountVectorizer(inputCol="filtered_tokens", outputCol="features")

# Step 6: Index Sentiment Labels
label_indexer = StringIndexer(inputCol="sentiment", outputCol="label")

# Step 7: Logistic Regression Classifier
classifier = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# Step 8: Create a Pipeline
pipeline = Pipeline(stages=[tokenizer, vectorizer, label_indexer, classifier])

# Step 9: Split Data into Training and Testing Sets
train_data, test_data = sentiment_data.randomSplit([0.8, 0.2], seed=42)

# Step 10: Train the Model
model = pipeline.fit(train_data)

# Step 11: Evaluate the Model
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Step 12: Optional - Display Sample Predictions
predictions.select("sentiment", "prediction").show(10)


24/12/02 14:53:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


SparkSession initialized successfully!


24/12/02 14:53:54 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:13 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:13 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:34 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:34 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:35 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:35 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:36 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:36 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:37 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:37 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/12/02 14:54:38 WARN DAGScheduler: Broadcasting larg

Accuracy: 0.8369642185578328


+---------+----------+
|sentiment|prediction|
+---------+----------+
| Positive|       0.0|
| Positive|       0.0|
| Negative|       1.0|
| Positive|       0.0|
| Positive|       0.0|
| Positive|       0.0|
| Negative|       2.0|
| Positive|       0.0|
| Negative|       0.0|
| Positive|       0.0|
+---------+----------+
only showing top 10 rows



In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("AmazonApparelAnalysis") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print("SparkSession initialized successfully!")

# Step 2: Load and Preprocess Data
data_path = "final_combined_dataset.csv"  # Replace with your dataset path
raw_data = spark.read.csv(data_path, header=True, inferSchema=True)

# Step 3: Handle Missing or Invalid Data
# Replace null values in critical columns with placeholders
cleaned_data = raw_data.fillna({
    "product_id": -1,
    "customer_id": -1,
    "star_rating": 0
})

# Remove rows with invalid placeholder values
cleaned_data = cleaned_data.filter(
    (col("product_id") != -1) & (col("customer_id") != -1) & (col("star_rating") != 0)
)

# Convert columns to appropriate data types
cleaned_data = cleaned_data.withColumn("customer_id", col("customer_id").cast(IntegerType()))
cleaned_data = cleaned_data.withColumn("product_id", col("product_id").cast(IntegerType()))
cleaned_data = cleaned_data.withColumn("star_rating", col("star_rating").cast(IntegerType()))

# Verify no null or invalid values remain
null_check = cleaned_data.filter(
    (col("product_id").isNull()) | (col("customer_id").isNull()) | (col("star_rating").isNull())
).count()

if null_check > 0:
    raise ValueError("Null values remain in critical columns after preprocessing.")

# Step 4: Check for Sufficient Variability
unique_customers = cleaned_data.select("customer_id").distinct().count()
unique_products = cleaned_data.select("product_id").distinct().count()

if unique_customers <= 1 or unique_products <= 1:
    raise ValueError("Not enough unique customers or products for ALS to work.")

# Step 5: Split Data into Training and Testing Sets
(training, test) = cleaned_data.randomSplit([0.8, 0.2], seed=42)

# Step 6: Ensure Overlap Between Training and Test
train_users = training.select("customer_id").distinct()
test_users = test.select("customer_id").distinct()

train_products = training.select("product_id").distinct()
test_products = test.select("product_id").distinct()

user_overlap = train_users.join(test_users, "customer_id").count()
product_overlap = train_products.join(test_products, "product_id").count()

print(f"User overlap between train and test: {user_overlap}")
print(f"Product overlap between train and test: {product_overlap}")

# Filter test set for existing users and products
filtered_test = test.join(train_users, "customer_id").join(train_products, "product_id")
print(f"Filtered test count: {filtered_test.count()}")

# Step 7: Initialize ALS Model
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="customer_id",
    itemCol="product_id",
    ratingCol="star_rating",
    coldStartStrategy="drop"
)

# Step 8: Train ALS Model
als_model = als.fit(training)

# Step 9: Make Predictions and Evaluate
predictions = als_model.transform(filtered_test)

if predictions.count() > 0:
    evaluator = RegressionEvaluator(
        metricName="rmse",
        labelCol="star_rating",
        predictionCol="prediction"
    )
    rmse = evaluator.evaluate(predictions)
    print(f"Root-mean-square error (RMSE): {rmse}")
else:
    print("No predictions available for evaluation.")

# Step 10: Generate Recommendations
# Recommend products for all users
user_recommendations = als_model.recommendForAllUsers(10)
user_recommendations.show(truncate=False)

# Recommend users for all products
product_recommendations = als_model.recommendForAllItems(10)
product_recommendations.show(truncate=False)

# Optional: Save the ALS Model
als_model.save("/Users/arunajithesh/Downloads/path_to_save_als_model")

# Stop the Spark Session
spark.stop()


SparkSession initialized successfully!


User overlap between train and test: 0
Product overlap between train and test: 9


Filtered test count: 0


No predictions available for evaluation.
+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|customer_id|recommendations                                                                                                                                                                                                                                          |
+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|48092640   |[{32050, 3.9463}, {1607056992, 2.2803218}, {12746525, 1.7839125}, {1465014578, 1.5355669}, {1439716897, 1.2710639}, {973259361, 1.1812987}, {1608322610, 1

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("AmazonApparelAnalysis") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print("SparkSession initialized successfully!")

# Step 2: Load and Preprocess Data
data_path = "final_combined_dataset.csv"  # Replace with your dataset path
raw_data = spark.read.csv(data_path, header=True, inferSchema=True)

# Step 3: Handle Missing or Invalid Data
cleaned_data = raw_data.fillna({
    "product_id": -1,
    "customer_id": -1,
    "star_rating": 0,
    "review_body": ""  # Assume "review_body" column holds the reviews
})

cleaned_data = cleaned_data.filter(
    (col("product_id") != -1) & (col("customer_id") != -1) & (col("star_rating") != 0)
)

cleaned_data = cleaned_data.withColumn("customer_id", col("customer_id").cast(IntegerType()))
cleaned_data = cleaned_data.withColumn("product_id", col("product_id").cast(IntegerType()))
cleaned_data = cleaned_data.withColumn("star_rating", col("star_rating").cast(IntegerType()))

null_check = cleaned_data.filter(
    (col("product_id").isNull()) | (col("customer_id").isNull()) | (col("star_rating").isNull())
).count()

if null_check > 0:
    raise ValueError("Null values remain in critical columns after preprocessing.")

# Step 4: Check for Sufficient Variability
unique_customers = cleaned_data.select("customer_id").distinct().count()
unique_products = cleaned_data.select("product_id").distinct().count()

if unique_customers <= 1 or unique_products <= 1:
    raise ValueError("Not enough unique customers or products for ALS to work.")

# Step 5: Split Data into Training and Testing Sets
(training, test) = cleaned_data.randomSplit([0.8, 0.2], seed=42)

# Step 6: Ensure Overlap Between Training and Test
train_users = training.select("customer_id").distinct()
test_users = test.select("customer_id").distinct()
train_products = training.select("product_id").distinct()
test_products = test.select("product_id").distinct()

user_overlap = train_users.join(test_users, "customer_id").count()
product_overlap = train_products.join(test_products, "product_id").count()

print(f"User overlap between train and test: {user_overlap}")
print(f"Product overlap between train and test: {product_overlap}")

filtered_test = test.join(train_users, "customer_id").join(train_products, "product_id")
print(f"Filtered test count: {filtered_test.count()}")

# Step 7: Initialize ALS Model
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="customer_id",
    itemCol="product_id",
    ratingCol="star_rating",
    coldStartStrategy="drop"
)

# Step 8: Train ALS Model
als_model = als.fit(training)

# Step 9: Make Predictions and Evaluate
predictions = als_model.transform(filtered_test)

if predictions.count() > 0:
    evaluator = RegressionEvaluator(
        metricName="rmse",
        labelCol="star_rating",
        predictionCol="prediction"
    )
    rmse = evaluator.evaluate(predictions)
    print(f"Root-mean-square error (RMSE): {rmse}")
else:
    print("No predictions available for evaluation.")

# Step 10: Generate Recommendations
user_recommendations = als_model.recommendForAllUsers(10)
product_recommendations = als_model.recommendForAllItems(10)

# Include Reviews in Recommendations
user_recommendations_with_reviews = user_recommendations \
    .withColumn("recommendations", col("recommendations").cast("array<struct<product_id:int,rating:float>>")) \
    .select("customer_id", "recommendations")

# Explode recommendations to join with reviews
from pyspark.sql.functions import explode
user_recommendations_with_reviews = user_recommendations_with_reviews \
    .withColumn("recommendation", explode(col("recommendations"))) \
    .select(
        col("customer_id"),
        col("recommendation.product_id").alias("product_id"),
        col("recommendation.rating").alias("predicted_rating")
    )

# Join with the cleaned data to fetch reviews
user_recommendations_with_reviews = user_recommendations_with_reviews \
    .join(cleaned_data.select("product_id", "review_body"), "product_id", "left")

user_recommendations_with_reviews.show(truncate=False)

# Optional: Save the ALS Model
als_model.save("path_to_save_als_model_new")

# Stop the Spark Session
spark.stop()


SparkSession initialized successfully!


User overlap between train and test: 0
Product overlap between train and test: 9


Filtered test count: 0


No predictions available for evaluation.


+----------+-----------+----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|customer_id|predicted_rating|review_body                                                                                                                                                                                                                                                                                                                                 |
+----------+-----------+----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

1. Data Preparation:
- Extracted review text from the dataset.
- Handled missing or null values in review data.

2. Text Preprocessing:
- Lowercased all text.
- Removed special characters, punctuation, and stopwords.
- Tokenized text into individual words.
- Applied stemming or lemmatization.

3. Feature Extraction:
- Converted text into numerical features using techniques like TF-IDF or Bag-of-Words.

4. Model Training:
- Split data into training and testing sets.
- Used a machine learning model (e.g., Logistic Regression, Naive Bayes) or a pre-trained NLP model to classify sentiments.

5. Model Evaluation:
- Tested model performance using metrics like accuracy, precision, recall, and F1-score.

6. Prediction:
- Classified each review into Positive, Neutral, or Negative sentiment categories.

1. Data Preparation:
- Extracted customer-product interaction data (e.g., ratings, purchases).
- Handled missing values by imputing or removing incomplete entries.

2. Data Transformation:
- Converted the dataset into a user-item interaction matrix.
- Normalized ratings or interactions to standardize values.

3. Model Initialization:
- Defined ALS parameters (e.g., rank, regularization, number of iterations).
- Split the data into training and validation sets.

4. Model Training:
- Trained the ALS model on the user-item matrix to identify latent factors.
- Iteratively minimized the loss function to improve recommendations.

5. Model Evaluation:
- Evaluated the model using metrics like Root Mean Square Error (RMSE) or Mean Absolute Error (MAE).
- Validated the model's performance on unseen data.

6. Prediction:
- Predicted customer preferences by generating a ranked list of products for each user.
- Recommended top-rated products based on the predicted ratings.

7. Output and app.py:
- Generated personalized recommendations for specific Customer IDs (e.g., 21291540).
- Visualized the recommended product list with associated ratings in the working sample demo.